In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lit
from pyspark.ml.feature import HashingTF, IDF, StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# 1️⃣ Khởi tạo Spark Session
spark = SparkSession.builder.appName("YelpHybridRecommendationTFIDF").getOrCreate()

# 2️⃣ Load dataset (Giả sử có 3 file: users, businesses, reviews)
businesses = spark.read.json("yelp_academic_dataset_business.json")
reviews = spark.read.json("yelp_academic_dataset_review.json")

# Chọn các cột quan trọng
businesses = businesses.select("business_id", "categories", "stars", "review_count")
reviews = reviews.select("user_id", "business_id", "stars")

# 3️⃣ Xử lý Content-Based Filtering với TF-IDF
# Tách categories thành danh sách
businesses = businesses.withColumn("categories", split(col("categories"), ", "))

# HashingTF để chuyển đổi categories thành vector TF
hashing_tf = HashingTF(inputCol="categories", outputCol="tf_features", numFeatures=1000)
businesses = hashing_tf.transform(businesses)

# Tính IDF để điều chỉnh trọng số từ
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
idf_model = idf.fit(businesses)
businesses = idf_model.transform(businesses)

# 4️⃣ Collaborative Filtering (CF) bằng ALS
# Chuyển đổi dữ liệu Review thành Spark DataFrame
reviews = reviews.withColumn("stars", col("stars").cast("float"))

# Index hóa user_id và business_id
user_indexer = StringIndexer(inputCol="user_id", outputCol="user_index")
business_indexer = StringIndexer(inputCol="business_id", outputCol="business_index")

reviews = user_indexer.fit(reviews).transform(reviews)
reviews = business_indexer.fit(reviews).transform(reviews)

# Xây dựng mô hình ALS
als = ALS(
    userCol="user_index",
    itemCol="business_index",
    ratingCol="stars",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop"
)

# Train mô hình ALS
als_model = als.fit(reviews)

# 5️⃣ Kết hợp Hybrid Filtering
# Hàm gợi ý dựa trên Collaborative Filtering
def get_cf_recommendations(user_id, top_n=10):
    user_df = spark.createDataFrame([(user_id,)], ["user_id"])
    recommendations = als_model.recommendForUserSubset(user_df, top_n)
    recommendations = recommendations.withColumn("recommendations", explode("recommendations"))
    recommendations = recommendations.select("user_id", col("recommendations.business_id"), col("recommendations.rating"))
    return recommendations

# Hàm gợi ý dựa trên Content-Based Filtering với TF-IDF
def get_cbf_recommendations(business_id, top_n=10):
    target_tfidf = businesses.filter(col("business_id") == business_id).select("tfidf_features").collect()[0][0]
    businesses_with_sim = businesses.withColumn("similarity", lit(target_tfidf.dot(col("tfidf_features"))))
    return businesses_with_sim.orderBy(col("similarity").desc()).limit(top_n)

# Hàm gợi ý dựa trên Hybrid Filtering
def get_hybrid_recommendations(user_id, business_id, alpha=0.5, top_n=10):
    cf_recs = get_cf_recommendations(user_id, top_n)
    cbf_recs = get_cbf_recommendations(business_id, top_n)

    # Gán trọng số
    cf_recs = cf_recs.withColumn("score", col("rating") * alpha)
    cbf_recs = cbf_recs.withColumn("score", col("stars") * (1 - alpha))

    # Gộp hai danh sách
    hybrid_recs = cf_recs.union(cbf_recs).groupBy("business_id").agg({"score": "sum"})
    hybrid_recs = hybrid_recs.orderBy(col("sum(score)").desc()).limit(top_n)

    return hybrid_recs

# 6️⃣ Đánh giá mô hình
# RMSE cho CF
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(als_model.transform(reviews))
print(f"RMSE của mô hình CF: {rmse:.4f}")

# 🛠️ Chạy thử nghiệm hệ thống Hybrid với TF-IDF
user_sample = "user_id_123"
business_sample = "business_id_456"
recommendations = get_hybrid_recommendations(user_sample, business_sample, alpha=0.6, top_n=10)
recommendations.show()
